In [1]:
import pandas as pd
import numpy as np
import csv
import pickle
from difflib import SequenceMatcher

### US SPONSORS & ENROLMENT DATA

In [2]:
### us_sponsors
df_us_sponsors = pd.read_csv('../trial_sponsors/usa_sponsor.csv', sep='|')
# Determining duplicates
mask = df_us_sponsors.duplicated(keep=False)
double = df_us_sponsors[mask]  #len(double) = 0 ==> no duplicates in df_us_years

### us_enrolment
df_us_enrolment = pd.read_csv('../trial_enrolment/us_enrolment.csv', sep='|')
# Determining duplicates
mask = df_us_enrolment.duplicated(keep=False)
double = df_us_enrolment[mask]  #len(double) = 0 ==> no duplicates in df_us_years

### merge 
df_us = pd.merge(df_us_sponsors, df_us_enrolment, on='trial_id')
df_us[0:3]

,trial_id,primary_spons,sponsor_type,enrolment
0,NCT03260985,Stanford University,Non-Commercial,75.0
1,NCT03268473,Bezmialem Vakif University,Non-Commercial,60.0
2,NCT03262610,"Rhythm Pharmaceuticals, Inc.",Commercial,NaN


In [3]:
len(df_us)

268859

### EU SPONSORS & ENROLMENT DATA

In [4]:
### eu_sponsors
df_eu_sponsors = pd.read_csv('../trial_sponsors/eu_sponsor.csv', sep='|')
# Determining duplicates
mask = df_eu_sponsors.duplicated(keep=False)
double = df_eu_sponsors[mask]  #len(double) = 0 ==> no duplicates in df_us_years

### eu_enrolment
df_eu_enrolment = pd.read_csv('../trial_enrolment/eu_enrolment.csv', sep='|')
# Determining duplicates
mask = df_eu_enrolment.duplicated(keep=False)
double = df_eu_enrolment[mask]  #len(double) = 0 ==> no duplicates in df_us_years

### merge
df_eu = pd.merge(df_eu_sponsors, df_eu_enrolment, on='trial_id')
df_eu[0:3]

,trial_id,primary_spons1,sponsor_type1,primary_spons2,sponsor_type2,primary_spons3,sponsor_type3,primary_spons4,sponsor_type4,enrolment
0,2004-000007-18,GlaxoSmithKline AB,Commercial,NaN,NaN,NaN,NaN,NaN,NaN,1110.0
1,2004-000012-13,Wyeth Research Division of Wyeth Pharmaceutica...,Commercial,NaN,NaN,NaN,NaN,NaN,NaN,360.0
2,2004-000015-25,Wyeth Research Division of Wyeth Pharmaceutica...,Commercial,NaN,NaN,NaN,NaN,NaN,NaN,1236.0


In [5]:
df_eu = df_eu.rename(index=str, columns={"primary_spons1": "primary_spons", 
                                         "sponsor_type1": "prim_sponsor_type", 
                                         "primary_spons2": "secondary_spons1", 
                                         "sponsor_type2": "sec_sponsor_type1",
                                         "primary_spons3": "secondary_spons2",
                                         "sponsor_type3": "sec_sponsor_type2",
                                         "primary_spons4": "secondary_spons3" ,
                                         "sponsor_type4": "sec_sponsor_type3",})
df_eu[0:1]

,trial_id,primary_spons,prim_sponsor_type,secondary_spons1,sec_sponsor_type1,secondary_spons2,sec_sponsor_type2,secondary_spons3,sec_sponsor_type3,enrolment
0,2004-000007-18,GlaxoSmithKline AB,Commercial,NaN,NaN,NaN,NaN,NaN,NaN,1110.0


In [6]:
len(df_eu)

32237

### ICTRP SPONSORS & ENROLMENT DATA

In [7]:
### ictrp_sponsors
df_ictrp_sponsors = pd.read_csv('../trial_sponsors/ictrp_sponsor.csv', sep='|')
# Determining duplicates
mask = df_ictrp_sponsors.duplicated(keep=False)
double = df_ictrp_sponsors[mask]  #len(double) = 0 ==> no duplicates in df_us_years

### ictrp_enrolment
df_ictrp_enrolment = pd.read_csv('../trial_enrolment/ictrp_enrolment.csv', sep='|')
# Determining duplicates
mask = df_ictrp_enrolment.duplicated(keep=False)
double = df_ictrp_enrolment[mask]  #len(double) = 0 ==> no duplicates in df_us_years

### merge
df_ictrp = pd.merge(df_ictrp_sponsors, df_ictrp_enrolment, on='trial_id')
df_ictrp[0:3]

,trial_id,primary_spons,sponsor_type,enrolment
0,ACTRN12605000058673,Commercial sector/Industry Pharmalab,Commercial,190.0
1,ACTRN12607000355471,Individual Maree Teesson,Commercial,1858.0
2,ACTRN12609000649213,Individual Professor Jon Currie,Commercial,56.0


In [8]:
len(df_ictrp)

114137

In [9]:
#To make it possible to compare sponsor_type
#Industry == commercial 
#U.S.Fed == Non-commercial 
#NIH == Non-commercial 

for i in range(0,len(df_us_sponsors)):
    if (df_us_sponsors.sponsor_type[i] == "Industry") or (df_us_sponsors.sponsor_type[i] == "industry") :
        df_us_sponsors.sponsor_type[i] = "Commercial"

for i in range(0,len(df_us_sponsors)):
    if (df_us_sponsors.sponsor_type[i] == "U.S.Fed") or (df_us_sponsors.sponsor_type[i] == "NIH") or (df_us_sponsors.sponsor_type[i] == "U.S. Fed") :
        df_us_sponsors.sponsor_type[i] = "Non-Commercial"
        

In [10]:
df_us_sponsors.sponsor_type[0:2]

0    Non-Commercial
1    Non-Commercial
Name: sponsor_type, dtype: object

### FUNCTIONS

In [11]:
### calculate similarity with "SequenceMatcher" where space isJunk 
def similar(a, b):
    return SequenceMatcher(lambda x: x == " ", a.lower(), b.lower()).ratio()

In [12]:
def is_nan(x):
    return (x is np.nan or x != x)

In [13]:
### Comparing the "Sponsor Type" of resulting lists
def list_sponsor_types(l1,l2):
    new_list = []
    for a, b in zip(l1,l2):
        
        if (is_nan(a) or is_nan(b)):
            new_list.append(np.nan)
        elif (a == b):
            new_list.append(1)
        else: 
            new_list.append(0)
            
    return(new_list)    

In [14]:
### Comparing the "Sponsor Name" of resulting lists
def list_sponsor_names(l1,l2):
    new_list = []
    for a, b in zip(l1, l2):
        if (is_nan(a) or is_nan(b)) :
            new_list.append(np.nan)
        else : 
            sim = similar(a, b)
            new_list.append(round(sim, 5))
    return(new_list)     

In [15]:
### Comparing the "enrolments" of resulting lists
def list_enrolments(l1,l2):
    new_list = []
    for a, b in zip(l1, l2):
        if (is_nan(a) or is_nan(b)) :
            new_list.append(np.nan)
        else : 
            diff = abs(a - b)
            new_list.append(diff)

    return(new_list)

## 01- Working for EU and US pairs

#### EU_US (ltitles_years) DATA

In [16]:
df_eu_us_ltitles_years = pd.read_excel('../_results/01-years_ltitles/new excels/eu_us_years_ltitles_list.xlsx')
df_eu_us = df_eu_us_ltitles_years.rename(index=str, columns={"similarity": "title_similarity"})
df_eu_us[0:3]

,id_EU,year_EU,long_title_EU,id_US,year_US,long_title_US,title_similarity
0,2004-000012-13,2004.0,"A Double-Blind, Placebo-Controlled, Parallel, ...",NCT00095342,NaN,"A Double-Blind, Placebo-Controlled, Parallel, ...",1
1,2004-000012-13,2004.0,"A Double-Blind, Placebo-Controlled, Parallel, ...",NCT00076206,2003.0,"A Double-Blind, Parallel, Placebo-Controlled, ...",0.809783
2,2004-000012-13,2004.0,"A Double-Blind, Placebo-Controlled, Parallel, ...",NCT00141830,2005.0,"A Randomized, Parallel, Double-Blind, Placebo-...",0.812865


In [17]:
len(df_eu_us)

17306

In [18]:
#Create lists of enrolment, sponsor type & name for the pairs 
#then calculate the similarity and the diffrence 
#and then inject the results in the previous dataframe

In [43]:
#Create lists of enrolment, sponsor type & name for the pairs 
list_EU_prim_sponsor_type = []
list_EU_sec_sponsor1_type = []
list_EU_sec_sponsor2_type = []
list_EU_sec_sponsor3_type = []

list_US_sponsor_type = []

list_EU_prim_sponsor_name = []
list_EU_sec_sponsor1_name = []
list_EU_sec_sponsor2_name = []
list_EU_sec_sponsor3_name = []

list_US_sponsor_name = []

list_EU_enrol = []
list_US_enrol = []

for i in range(0,len(df_eu_us)):
    id_EU = df_eu_us.id_EU[i]
    id_US = df_eu_us.id_US[i]
    
    for j in range(0,len(df_eu)): 
        if (id_EU == df_eu.trial_id[j]) : 
            
            
            list_EU_prim_sponsor_name.append(df_eu.primary_spons[j])
            list_EU_prim_sponsor_type.append(df_eu.prim_sponsor_type[j])
            
            list_EU_sec_sponsor1_name.append(df_eu.secondary_spons1[j])
            list_EU_sec_sponsor1_type.append(df_eu.sec_sponsor_type1[j])
            
            list_EU_sec_sponsor2_name.append(df_eu.secondary_spons2[j])
            list_EU_sec_sponsor2_type.append(df_eu.sec_sponsor_type2[j])
            
            list_EU_sec_sponsor3_name.append(df_eu.secondary_spons3[j])
            list_EU_sec_sponsor3_type.append(df_eu.sec_sponsor_type3[j])
            
            list_EU_enrol.append(df_eu.enrolment[j])
            
            print(i,"id_EU",id_EU)
            
    for k in range(0,len(df_us)):
        if (id_US == df_us.trial_id[k]) :
            
            list_US_sponsor_type.append(df_us.sponsor_type[k])
            list_US_sponsor_name.append(df_us.primary_spons[k])
            list_US_enrol.append(df_us.enrolment[k])
            
            print(i,"id_US:",id_US)
            print('-----------------------')


0 id_EU 2004-000012-13
0 id_US: NCT00095342
-----------------------
1 id_EU 2004-000012-13
1 id_US: NCT00076206
-----------------------
2 id_EU 2004-000012-13
2 id_US: NCT00141830
-----------------------
3 id_EU 2004-000015-25
3 id_US: NCT00083993
-----------------------
4 id_EU 2004-000020-32
4 id_US: NCT00095550
-----------------------
5 id_EU 2004-000020-32
5 id_US: NCT00095394
-----------------------
6 id_EU 2004-000022-75
6 id_US: NCT00095290
-----------------------
7 id_EU 2004-000023-15
7 id_US: NCT00097591
-----------------------
8 id_EU 2004-000028-34
8 id_US: NCT00087711
-----------------------
9 id_EU 2004-000029-31
9 id_US: NCT00287716
-----------------------
10 id_EU 2004-000029-31
10 id_US: NCT00287729
-----------------------
11 id_EU 2004-000033-11
11 id_US: NCT00095550
-----------------------
12 id_EU 2004-000033-11
12 id_US: NCT00095394
-----------------------
13 id_EU 2004-000039-27
13 id_US: NCT00129766
-----------------------
14 id_EU 2004-000046-21
14 id_US: NCT002

117 id_EU 2004-000404-40
117 id_US: NCT00473460
-----------------------
118 id_EU 2004-000405-22
118 id_US: NCT00301782
-----------------------
119 id_EU 2004-000406-51
119 id_US: NCT00563134
-----------------------
120 id_EU 2004-000406-51
120 id_US: NCT00089986
-----------------------
121 id_EU 2004-000408-41
121 id_US: NCT00093093
-----------------------
122 id_EU 2004-000408-41
122 id_US: NCT00230958
-----------------------
123 id_EU 2004-000424-32
123 id_US: NCT00083343
-----------------------
124 id_EU 2004-000432-85
124 id_US: NCT00124436
-----------------------
125 id_EU 2004-000438-35
125 id_US: NCT00081458
-----------------------
126 id_EU 2004-000440-24
126 id_US: NCT00151736
-----------------------
127 id_EU 2004-000441-38
127 id_US: NCT00377169
-----------------------
128 id_EU 2004-000441-38
128 id_US: NCT00107679
-----------------------
129 id_EU 2004-000441-38
129 id_US: NCT00027352
-----------------------
130 id_EU 2004-000442-21
130 id_US: NCT00160134
----------------

231 id_EU 2004-000804-41
231 id_US: NCT00291850
-----------------------
232 id_EU 2004-000805-23
232 id_US: NCT00169585
-----------------------
233 id_EU 2004-000806-39
233 id_US: NCT00286819
-----------------------
234 id_EU 2004-000816-24
234 id_US: NCT00234923
-----------------------
235 id_EU 2004-000817-20
235 id_US: NCT00199394
-----------------------
236 id_EU 2004-000821-31
236 id_US: NCT00185354
-----------------------
237 id_EU 2004-000822-58
237 id_US: NCT00174759
-----------------------
238 id_EU 2004-000825-31
238 id_US: NCT00163462
-----------------------
239 id_EU 2004-000830-35
239 id_US: NCT01888107
-----------------------
240 id_EU 2004-000833-12
240 id_US: NCT00643045
-----------------------
241 id_EU 2004-000835-27
241 id_US: NCT00642889
-----------------------
242 id_EU 2004-000848-24
242 id_US: NCT00101985
-----------------------
243 id_EU 2004-000851-41
243 id_US: NCT00239174
-----------------------
244 id_EU 2004-000867-92
244 id_US: NCT00094757
----------------

345 id_EU 2004-001457-28
345 id_US: NCT00291837
-----------------------
346 id_EU 2004-001458-10
346 id_US: NCT00984386
-----------------------
347 id_EU 2004-001460-42
347 id_US: NCT00174694
-----------------------
348 id_EU 2004-001467-22
348 id_US: NCT00092001
-----------------------
349 id_EU 2004-001482-17
349 id_US: NCT00094549
-----------------------
350 id_EU 2004-001485-41
350 id_US: NCT00129181
-----------------------
351 id_EU 2004-001487-43
351 id_US: NCT00312481
-----------------------
352 id_EU 2004-001490-26
352 id_US: NCT00747214
-----------------------
353 id_EU 2004-001491-39
353 id_US: NCT00105144
-----------------------
354 id_EU 2004-001497-13
354 id_US: NCT00644176
-----------------------
355 id_EU 2004-001500-13
355 id_US: NCT00151450
-----------------------
356 id_EU 2004-001540-71
356 id_US: NCT00500656
-----------------------
357 id_EU 2004-001540-71
357 id_US: NCT00097695
-----------------------
358 id_EU 2004-001548-60
358 id_US: NCT00081653
----------------

459 id_EU 2004-002163-24
459 id_US: NCT00195715
-----------------------
460 id_EU 2004-002165-20
460 id_US: NCT00254267
-----------------------
461 id_EU 2004-002165-20
461 id_US: NCT00089960
-----------------------
462 id_EU 2004-002166-38
462 id_US: NCT00152984
-----------------------
463 id_EU 2004-002171-16
463 id_US: NCT00395772
-----------------------
464 id_EU 2004-002172-42
464 id_US: NCT00492635
-----------------------
465 id_EU 2004-002174-32
465 id_US: NCT00196755
-----------------------
466 id_EU 2004-002187-22
466 id_US: NCT00120003
-----------------------
467 id_EU 2004-002203-32
467 id_US: NCT00283062
-----------------------
468 id_EU 2004-002216-28
468 id_US: NCT00163098
-----------------------
469 id_EU 2004-002231-92
469 id_US: NCT00093080
-----------------------
470 id_EU 2004-002236-26
470 id_US: NCT00423488
-----------------------
471 id_EU 2004-002245-12
471 id_US: NCT00110123
-----------------------
472 id_EU 2004-002252-33
472 id_US: NCT00262236
----------------

...

16824 id_EU 2016-004990-42
16824 id_US: NCT03263715
-----------------------
16825 id_EU 2016-004991-23
16825 id_US: NCT03410953
-----------------------
16826 id_EU 2016-004996-33
16826 id_US: NCT02823470
-----------------------
16827 id_EU 2016-004997-16
16827 id_US: NCT03345914
-----------------------
16828 id_EU 2016-004997-16
16828 id_US: NCT03054428
-----------------------
16829 id_EU 2016-005004-26
16829 id_US: NCT03201003
-----------------------
16830 id_EU 2016-005008-24
16830 id_US: NCT03336073
-----------------------
16831 id_EU 2016-005017-45
16831 id_US: NCT03241108
-----------------------
16832 id_EU 2016-005020-29
16832 id_US: NCT03304379
-----------------------
16833 id_EU 2016-005020-29
16833 id_US: NCT03285646
-----------------------
16834 id_EU 2016-005020-29
16834 id_US: NCT03161093
-----------------------
16835 id_EU 2016-005032-14
16835 id_US: NCT03158948
-----------------------
16836 id_EU 2016-005035-33
16836 id_US: NCT03188666
-----------------------
16837 id_EU 

16932 id_EU 2017-000438-79
16932 id_US: NCT03268226
-----------------------
16933 id_EU 2017-000439-14
16933 id_US: NCT03236857
-----------------------
16934 id_EU 2017-000456-26
16934 id_US: NCT03251612
-----------------------
16935 id_EU 2017-000469-62
16935 id_US: NCT03360682
-----------------------
16936 id_EU 2017-000475-90
16936 id_US: NCT03395639
-----------------------
16937 id_EU 2017-000497-11
16937 id_US: NCT03187769
-----------------------
16938 id_EU 2017-000505-20
16938 id_US: NCT03125941
-----------------------
16939 id_EU 2017-000506-37
16939 id_US: NCT03466463
-----------------------
16940 id_EU 2017-000508-92
16940 id_US: NCT03145961
-----------------------
16941 id_EU 2017-000538-78
16941 id_US: NCT03139604
-----------------------
16942 id_EU 2017-000540-18
16942 id_US: NCT03150719
-----------------------
16943 id_EU 2017-000544-17
16943 id_US: NCT03125941
-----------------------
16944 id_EU 2017-000545-52
16944 id_US: NCT03320876
-----------------------
16945 id_EU 

17040 id_EU 2017-001156-55
17040 id_US: NCT03248882
-----------------------
17041 id_EU 2017-001163-20
17041 id_US: NCT03444324
-----------------------
17042 id_EU 2017-001176-31
17042 id_US: NCT03269175
-----------------------
17043 id_EU 2017-001189-98
17043 id_US: NCT03348969
-----------------------
17044 id_EU 2017-001203-79
17044 id_US: NCT03252587
-----------------------
17045 id_EU 2017-001203-79
17045 id_US: NCT02975336
-----------------------
17046 id_EU 2017-001203-79
17046 id_US: NCT03161483
-----------------------
17047 id_EU 2017-001219-35
17047 id_US: NCT03277690
-----------------------
17048 id_EU 2017-001224-22
17048 id_US: NCT03222973
-----------------------
17049 id_EU 2017-001225-41
17049 id_US: NCT03345823
-----------------------
17050 id_EU 2017-001226-18
17050 id_US: NCT03345849
-----------------------
17051 id_EU 2017-001226-18
17051 id_US: NCT03345836
-----------------------
17052 id_EU 2017-001239-38
17052 id_US: NCT03301051
-----------------------
17053 id_EU 

17148 id_EU 2017-002025-38
17148 id_US: NCT03244800
-----------------------
17149 id_EU 2017-002046-71
17149 id_US: NCT03205150
-----------------------
17150 id_EU 2017-002050-36
17150 id_US: NCT03187743
-----------------------
17151 id_EU 2017-002065-21
17151 id_US: NCT03363854
-----------------------
17152 id_EU 2017-002065-21
17152 id_US: NCT03131648
-----------------------
17153 id_EU 2017-002065-21
17153 id_US: NCT03160885
-----------------------
17154 id_EU 2017-002088-16
17154 id_US: NCT03373071
-----------------------
17155 id_EU 2017-002093-40
17155 id_US: NCT03319641
-----------------------
17156 id_EU 2017-002113-64
17156 id_US: NCT03300427
-----------------------
17157 id_EU 2017-002153-11
17157 id_US: NCT03406104
-----------------------
17158 id_EU 2017-002156-84
17158 id_US: NCT03376321
-----------------------
17159 id_EU 2017-002156-84
17159 id_US: NCT03381196
-----------------------
17160 id_EU 2017-002165-21
17160 id_US: NCT03358030
-----------------------
17161 id_EU 

17256 id_EU 2017-003200-48
17256 id_US: NCT03373383
-----------------------
17257 id_EU 2017-003220-78
17257 id_US: NCT03341299
-----------------------
17258 id_EU 2017-003220-78
17258 id_US: NCT03465878
-----------------------
17259 id_EU 2017-003240-20
17259 id_US: NCT03409757
-----------------------
17260 id_EU 2017-003252-24
17260 id_US: NCT03379675
-----------------------
17261 id_EU 2017-003284-35
17261 id_US: NCT03368118
-----------------------
17262 id_EU 2017-003302-40
17262 id_US: NCT03452137
-----------------------
17263 id_EU 2017-003309-16
17263 id_US: NCT03339297
-----------------------
17264 id_EU 2017-003310-74
17264 id_US: NCT03412890
-----------------------
17265 id_EU 2017-003332-36
17265 id_US: NCT03384966
-----------------------
17266 id_EU 2017-003366-27
17266 id_US: NCT03298698
-----------------------
17267 id_EU 2017-003377-34
17267 id_US: NCT03384654
-----------------------
17268 id_EU 2017-003430-85
17268 id_US: NCT03374358
-----------------------
17269 id_EU 

In [44]:
#Sponsor Names
EUprim_USprim_sponsor_name = list_sponsor_names(list_EU_prim_sponsor_name, list_US_sponsor_name)
EUsec1_USprim_sponsor_name = list_sponsor_names(list_EU_sec_sponsor1_name, list_US_sponsor_name)
EUsec2_USprim_sponsor_name = list_sponsor_names(list_EU_sec_sponsor2_name, list_US_sponsor_name)
EUsec3_USprim_sponsor_name = list_sponsor_names(list_EU_sec_sponsor3_name, list_US_sponsor_name)

EUprim_USprim_sponsor_name[0:2]

[0.23529, 0.23529]

In [45]:
#Sponsor Types
EUprim_USprim_sponsor_type = list_sponsor_types(list_EU_prim_sponsor_type, list_US_sponsor_type)
EUsec1_USprim_sponsor_type = list_sponsor_types(list_EU_sec_sponsor1_type, list_US_sponsor_type)
EUsec2_USprim_sponsor_type = list_sponsor_types(list_EU_sec_sponsor2_type, list_US_sponsor_type)
EUsec3_USprim_sponsor_type = list_sponsor_types(list_EU_sec_sponsor3_type, list_US_sponsor_type)

EUsec2_USprim_sponsor_type[0:2]

[nan, nan]

In [46]:
#Enrolments
EU_US_enrol = list_enrolments(list_EU_enrol, list_US_enrol)
EU_US_enrol[0:2]

[30.0, nan]

In [47]:
#Save the final dataframe

df_eu_us['enrolment_EU'] = list_EU_enrol
df_eu_us['enrolment_US'] = list_US_enrol
df_eu_us['enrolment_diff'] = EU_US_enrol


df_eu_us['spon_US'] = list_US_sponsor_name

df_eu_us['primary_spon_EU'] = list_EU_prim_sponsor_name
df_eu_us['prim_spon_similarity'] = EUprim_USprim_sponsor_name

df_eu_us['secondary_spon1_EU'] = list_EU_sec_sponsor1_name
df_eu_us['sec_spon_similarity1'] = EUsec1_USprim_sponsor_name

df_eu_us['secondary_spon2_EU'] = list_EU_sec_sponsor2_name
df_eu_us['sec_spon_similarity2'] = EUsec2_USprim_sponsor_name

df_eu_us['secondary_spon3_EU'] = list_EU_sec_sponsor3_name
df_eu_us['sec_spon_similarity3'] = EUsec3_USprim_sponsor_name



df_eu_us['spon_type_US'] = list_US_sponsor_type

df_eu_us['primary_spon_type_EU'] = list_EU_prim_sponsor_type
df_eu_us['same_prim_spon_type'] = EUprim_USprim_sponsor_type

df_eu_us['secondary_spon1_type_EU'] = list_EU_sec_sponsor1_type
df_eu_us['same_sec_spon_type1'] = EUsec1_USprim_sponsor_type

df_eu_us['secondary_spon2_type_EU'] = list_EU_sec_sponsor2_type
df_eu_us['same_sec_spon_type2'] = EUsec2_USprim_sponsor_type

df_eu_us['secondary_spon3_type_EU'] = list_EU_sec_sponsor3_type
df_eu_us['same_sec_spon_type3'] = EUsec3_USprim_sponsor_type

In [48]:
df_eu_us[:2]

,id_EU,year_EU,long_title_EU,id_US,year_US,long_title_US,title_similarity,enrolment_EU,enrolment_US,enrolment_diff,...,sec_spon_similarity3,spon_type_US,primary_spon_type_EU,same_prim_spon_type,secondary_spon1_type_EU,same_sec_spon_type1,secondary_spon2_type_EU,same_sec_spon_type2,secondary_spon3_type_EU,same_sec_spon_type3
0,2004-000012-13,2004.0,"A Double-Blind, Placebo-Controlled, Parallel, ...",NCT00095342,NaN,"A Double-Blind, Placebo-Controlled, Parallel, ...",1,360.0,390.0,30.0,...,NaN,Commercial,Commercial,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2004-000012-13,2004.0,"A Double-Blind, Placebo-Controlled, Parallel, ...",NCT00076206,2003.0,"A Double-Blind, Parallel, Placebo-Controlled, ...",0.809783,360.0,NaN,NaN,...,NaN,Commercial,Commercial,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
len(df_eu_us)

17306

In [50]:
df_eu_us.to_excel('../_results/02-sponsors_enrolments/NEW_final_eu_us.xlsx')

### TEST

In [110]:
df_eu_us_SUB = df_eu_us_ltitles_years[0:50]
df_eu_us_SUB[0:2]

,id_EU,year_EU,long_title_EU,id_US,year_US,long_title_US,similarity
0,2004-000012-13,2004.0,"A Double-Blind, Placebo-Controlled, Parallel, ...",NCT00095342,NaN,"A Double-Blind, Placebo-Controlled, Parallel, ...",0.875000
1,2004-000020-32,2005.0,The Efficacy and Safety of Irbesartan/HCTZ Com...,NCT00095394,2004.0,The Efficacy and Safety of Irbesartan/HCTZ Com...,0.817204


In [130]:
list_sponsor_types(list_EU_sponsor_type, list_US_sponsor_type)[0:2]

[1, 1, 1, 1, 1]

In [104]:
list_sponsor_names(list_EU_sponsor_name, list_US_sponsor_name)[0:2]

[0.23529, 0.57576, 0.60606, 0.84, 0.84, 0.33333, 0.33333, 0.57576, 0.57576, 0.64286, 0.64, 0.54545, 0.71429, 0.71429, 1.0, 1.0, 0.85714, 0.60606, 0.23729, 0.66667, 0.71429, 0.8, 0.65455, 0.66667, 0.59091, 0.16393, 0.36111, 0.84, 0.66667, 0.66667, 0.71429, 0.71429, 0.71429, 0.71429, 0.68, 0.24762, 0.28571, 0.28571, 0.19355, 0.8, 0.8, 0.8, 0.14286, 0.52381, 0.52381, 0.52381, 0.71429, 0.71429, 0.71429, 0.52381]


In [92]:
list_enrolments(list_EU_enrol, list_US_enrol)[0:2]

[30.0, 181.0, 215.0, 619.0, 13.0, 165.0, 256.0, 0.0, 149.0, 35.0, 9.0, 285.0, 1.0, 44.0, 34.0, 0.0, 448.0, 'NaN', 1094.0, 38.0, 7537.0, 0.0, 0.0, 1.0, 35.0, 201.0, 269.0, 14.0, 'NaN', 284.0, 2.0, 3.0, 2.0, 3.0, 196.0, 175.0, 3.0, 0.0, 1.0, 322.0, 0.0, 0.0, 0.0, 441.0, 401.0, 53.0, 216.0, 176.0, 172.0, 79.0]


In [112]:
df_eu_us_SUB['spon_type_EU'] = list_EU_sponsor_type
df_eu_us_SUB['spon_type_US'] = list_US_sponsor_type
df_eu_us_SUB['same_spon_type'] = US_EU_sponsor_type

df_eu_us_SUB['sponsor_EU'] = list_EU_sponsor_name
df_eu_us_SUB['sponsor_US'] = list_US_sponsor_name
df_eu_us_SUB['spon_similarity'] = US_EU_sponsor_name

df_eu_us_SUB['enrolment_EU'] = list_EU_enrol
df_eu_us_SUB['enrolment_US'] = list_US_enrol
df_eu_us_SUB['enrolment_diff'] = US_EU_enrol

In [113]:
df_eu_us_SUB[0:2]

,id_EU,year_EU,long_title_EU,id_US,year_US,long_title_US,title_similarity,spon_type_EU,spon_type_US,same_spon_type,sponsor_EU,sponsor_US,spon_similarity,enrolment_EU,enrolment_US,enrolment_diff
0,2004-000012-13,2004.0,"A Double-Blind, Placebo-Controlled, Parallel, ...",NCT00095342,NaN,"A Double-Blind, Placebo-Controlled, Parallel, ...",0.875000,Commercial,Commercial,1,Wyeth Research Division of Wyeth Pharmaceutica...,Wyeth is now a wholly owned subsidiary of Pfizer,0.23529,360.0,390.0,30
1,2004-000020-32,2005.0,The Efficacy and Safety of Irbesartan/HCTZ Com...,NCT00095394,2004.0,The Efficacy and Safety of Irbesartan/HCTZ Com...,0.817204,Commercial,Commercial,1,Bristol Myers Squibb International Corporation,Bristol-Myers Squibb,0.57576,826.0,645.0,181


In [114]:
df_eu_us_SUB.to_excel('test_eu_us.xlsx')

## 02- Working for EU and ICTRP pairs

#### EU_ICTRP (ltitles_years) DATA

In [20]:
df_eu_ictrp_ltitles_years = pd.read_excel('../_results/01-years_ltitles/new excels/eu_ictrp_years_ltitles_list.xlsx')
df_eu_ictrp = df_eu_ictrp_ltitles_years.rename(index=str, columns={"similarity": "title_similarity"})
df_eu_ictrp[0:3]

,id_EU,year_EU,long_title_EU,id_ICTRP,year_ICTRP,long_title_ICTRP,title_similarity
110,2004-000029-31,2005.0,"A Randomized, Double-Blind, Placebo-Controlled...",PER-071-11,NaN,"A RANDOMIZED, DOUBLE-BLIND, PLACEBO- CONTROLLE...",0.868217
331,2004-000107-18,2006.0,A RANDOMISED OPEN COMPARISON OF INTRAVAGINAL A...,ISRCTN27617624,2005.0,A randomised open comparision of intravaginal ...,0.976378
332,2004-000107-18,2006.0,A RANDOMISED OPEN COMPARISON OF INTRAVAGINAL A...,ISRCTN36773557,2005.0,A randomised open comparison of intravaginal A...,0.957447


In [21]:
len(df_eu_ictrp)

1249

In [25]:
#Create lists of enrolment, sponsor type & name for the pairs 
list_EU_prim_sponsor_type = []
list_EU_sec_sponsor1_type = []
list_EU_sec_sponsor2_type = []
list_EU_sec_sponsor3_type = []

list_ICTRP_sponsor_type = []

list_EU_prim_sponsor_name = []
list_EU_sec_sponsor1_name = []
list_EU_sec_sponsor2_name = []
list_EU_sec_sponsor3_name = []

list_ICTRP_sponsor_name = []

list_EU_enrol = []
list_ICTRP_enrol = []

for i in range(0,len(df_eu_ictrp)):
    id_EU = df_eu_ictrp.id_EU[i]
    id_ICTRP = df_eu_ictrp.id_ICTRP[i]
    
    for j in range(0,len(df_eu)): 
        if (id_EU == df_eu.trial_id[j]) : 
            
            
            list_EU_prim_sponsor_name.append(df_eu.primary_spons[j])
            list_EU_prim_sponsor_type.append(df_eu.prim_sponsor_type[j])
            
            list_EU_sec_sponsor1_name.append(df_eu.secondary_spons1[j])
            list_EU_sec_sponsor1_type.append(df_eu.sec_sponsor_type1[j])
            
            list_EU_sec_sponsor2_name.append(df_eu.secondary_spons2[j])
            list_EU_sec_sponsor2_type.append(df_eu.sec_sponsor_type2[j])
            
            list_EU_sec_sponsor3_name.append(df_eu.secondary_spons3[j])
            list_EU_sec_sponsor3_type.append(df_eu.sec_sponsor_type3[j])
            
            list_EU_enrol.append(df_eu.enrolment[j])
            
            print(i,"id_EU",id_EU)
            
    for k in range(0,len(df_ictrp)):
        if (id_ICTRP == df_ictrp.trial_id[k]) :
            
            list_ICTRP_sponsor_type.append(df_ictrp.sponsor_type[k])
            list_ICTRP_sponsor_name.append(df_ictrp.primary_spons[k])
            list_ICTRP_enrol.append(df_ictrp.enrolment[k])
            
            print(i,"id_ICTRP:",id_ICTRP)
            print('-----------------------')


0 id_EU 2004-000029-31
0 id_ICTRP: PER-071-11
-----------------------
1 id_EU 2004-000107-18
1 id_ICTRP: ISRCTN27617624
-----------------------
2 id_EU 2004-000107-18
2 id_ICTRP: ISRCTN36773557
-----------------------
3 id_EU 2004-000108-42
3 id_ICTRP: ISRCTN27617624
-----------------------
4 id_EU 2004-000108-42
4 id_ICTRP: ISRCTN36773557
-----------------------
5 id_EU 2004-000629-32
5 id_ICTRP: ISRCTN45133151
-----------------------
6 id_EU 2004-000675-34
6 id_ICTRP: NTR217
-----------------------
7 id_EU 2004-000905-24
7 id_ICTRP: ISRCTN18994246
-----------------------
8 id_EU 2004-001083-43
8 id_ICTRP: ACTRN12605000601639
-----------------------
9 id_EU 2004-001290-25
9 id_ICTRP: ACTRN12605000141640
-----------------------
10 id_EU 2004-001647-30
10 id_ICTRP: DRKS00005309
-----------------------
11 id_EU 2004-001773-26
11 id_ICTRP: ACTRN12605000644662
-----------------------
12 id_EU 2004-001786-18
12 id_ICTRP: NTR355
-----------------------
13 id_EU 2004-002547-27
13 id_ICTRP: IS

108 id_EU 2006-002175-40
108 id_ICTRP: ISRCTN69541857
-----------------------
109 id_EU 2006-002180-72
109 id_ICTRP: NTR1602
-----------------------
110 id_EU 2006-002249-35
110 id_ICTRP: ISRCTN77940178
-----------------------
111 id_EU 2006-002337-20
111 id_ICTRP: NTR671
-----------------------
112 id_EU 2006-002342-11
112 id_ICTRP: ISRCTN93019988
-----------------------
113 id_EU 2006-002358-31
113 id_ICTRP: NTR1419
-----------------------
114 id_EU 2006-002392-40
114 id_ICTRP: NTR1216
-----------------------
115 id_EU 2006-002403-13
115 id_ICTRP: ACTRN12605000707662
-----------------------
116 id_EU 2006-002457-69
116 id_ICTRP: CTRI/2009/091/000916
-----------------------
117 id_EU 2006-002663-11
117 id_ICTRP: ISRCTN81647490
-----------------------
118 id_EU 2006-002770-22
118 id_ICTRP: ISRCTN80601080
-----------------------
119 id_EU 2006-002787-26
119 id_ICTRP: NTR1078
-----------------------
120 id_EU 2006-002886-38
120 id_ICTRP: ISRCTN95777824
-----------------------
121 id_EU 2

217 id_EU 2007-004579-21
217 id_ICTRP: NTR1147
-----------------------
218 id_EU 2007-004686-18
218 id_ICTRP: ISRCTN03110798
-----------------------
219 id_EU 2007-004689-42
219 id_ICTRP: ISRCTN21269614
-----------------------
220 id_EU 2007-004716-31
220 id_ICTRP: ISRCTN24648386
-----------------------
221 id_EU 2007-004716-31
221 id_ICTRP: NTR1038
-----------------------
222 id_EU 2007-004895-37
222 id_ICTRP: CTRI/2009/091/000450
-----------------------
223 id_EU 2007-004896-20
223 id_ICTRP: CTRI/2009/091/000450
-----------------------
224 id_EU 2007-004897-26
224 id_ICTRP: ISRCTN68407918
-----------------------
225 id_EU 2007-005080-10
225 id_ICTRP: ISRCTN45049889
-----------------------
226 id_EU 2007-005230-37
226 id_ICTRP: NTR1526
-----------------------
227 id_EU 2007-005232-81
227 id_ICTRP: NTR1246
-----------------------
228 id_EU 2007-005451-42
228 id_ICTRP: NTR1595
-----------------------
229 id_EU 2007-005487-28
229 id_ICTRP: NTR1461
-----------------------
230 id_EU 2007-0

326 id_ICTRP: CTRI/2012/05/002622
-----------------------
327 id_EU 2008-008442-21
327 id_ICTRP: NTR1701
-----------------------
328 id_EU 2008-008479-32
328 id_ICTRP: ISRCTN73579730
-----------------------
329 id_EU 2008-008899-14
329 id_ICTRP: ISRCTN76597892
-----------------------
330 id_EU 2008-008954-22
330 id_ICTRP: NTR1940
-----------------------
331 id_EU 2008-008974-30
331 id_ICTRP: CTRI/2009/091/000663
-----------------------
332 id_EU 2008-008974-30
332 id_ICTRP: CTRI/2010/091/000098
-----------------------
333 id_EU 2009-009334-32
333 id_ICTRP: NTR1779
-----------------------
334 id_EU 2009-009457-13
334 id_ICTRP: NTR1880
-----------------------
335 id_EU 2009-009596-35
335 id_ICTRP: ISRCTN04386758
-----------------------
336 id_EU 2009-009986-32
336 id_ICTRP: NTR1932
-----------------------
337 id_EU 2009-010083-42
337 id_ICTRP: ISRCTN07270894
-----------------------
338 id_EU 2009-010163-16
338 id_ICTRP: ISRCTN77283008
-----------------------
339 id_EU 2009-010335-41
339 

434 id_ICTRP: ISRCTN18217390
-----------------------
435 id_EU 2009-016000-22
435 id_ICTRP: ISRCTN55761467
-----------------------
436 id_EU 2009-016080-11
436 id_ICTRP: NTR2391
-----------------------
437 id_EU 2009-016159-23
437 id_ICTRP: NTR2898
-----------------------
438 id_EU 2009-016197-33
438 id_ICTRP: CTRI/2010/091/001181
-----------------------
439 id_EU 2009-016300-23
439 id_ICTRP: ISRCTN47146067
-----------------------
440 id_EU 2009-016364-36
440 id_ICTRP: NTR2516
-----------------------
441 id_EU 2009-016371-30
441 id_ICTRP: CTRI/2010/091/000335
-----------------------
442 id_EU 2009-016790-15
442 id_ICTRP: NTR1228
-----------------------
443 id_EU 2009-016790-15
443 id_ICTRP: NTR1495
-----------------------
444 id_EU 2009-016790-15
444 id_ICTRP: NTR1637
-----------------------
445 id_EU 2009-016790-15
445 id_ICTRP: NTR2158
-----------------------
446 id_EU 2009-016812-18
446 id_ICTRP: CTRI/2011/05/001713
-----------------------
447 id_EU 2009-016904-21
447 id_ICTRP: NTR2

...

760 id_ICTRP: NTR3526
-----------------------
761 id_EU 2012-000432-26
761 id_ICTRP: NTR4722
-----------------------
762 id_EU 2012-000438-21
762 id_ICTRP: ISRCTN65844716
-----------------------
763 id_EU 2012-000524-18
763 id_ICTRP: PER-125-12
-----------------------
764 id_EU 2012-000525-45
764 id_ICTRP: PER-125-12
-----------------------
765 id_EU 2012-000642-35
765 id_ICTRP: PER-098-12
-----------------------
766 id_EU 2012-000643-27
766 id_ICTRP: PER-102-12
-----------------------
767 id_EU 2012-000654-55
767 id_ICTRP: ISRCTN80658312
-----------------------
768 id_EU 2012-000671-16
768 id_ICTRP: PER-002-13
-----------------------
769 id_EU 2012-000712-29
769 id_ICTRP: ISRCTN25212012
-----------------------
770 id_EU 2012-000726-21
770 id_ICTRP: PER-119-12
-----------------------
771 id_EU 2012-000747-26
771 id_ICTRP: ISRCTN44502774
-----------------------
772 id_EU 2012-000793-30
772 id_ICTRP: NTR4720
-----------------------
773 id_EU 2012-000796-16
773 id_ICTRP: CTRI/2011/11/0020

870 id_EU 2013-000503-17
870 id_ICTRP: JPRN-JapicCTI-142578
-----------------------
871 id_EU 2013-000521-31
871 id_ICTRP: ISRCTN79871784
-----------------------
872 id_EU 2013-000582-36
872 id_ICTRP: PER-032-13
-----------------------
873 id_EU 2013-000594-69
873 id_ICTRP: PER-086-13
-----------------------
874 id_EU 2013-000603-16
874 id_ICTRP: NTR4308
-----------------------
875 id_EU 2013-000626-63
875 id_ICTRP: CTRI/2014/01/004317
-----------------------
876 id_EU 2013-000626-63
876 id_ICTRP: CTRI/2014/01/004329
-----------------------
877 id_EU 2013-000636-10
877 id_ICTRP: CTRI/2014/01/004317
-----------------------
878 id_EU 2013-000636-10
878 id_ICTRP: CTRI/2014/01/004329
-----------------------
879 id_EU 2013-000691-15
879 id_ICTRP: PER-020-14
-----------------------
880 id_EU 2013-000789-13
880 id_ICTRP: ISRCTN70595832
-----------------------
881 id_EU 2013-000801-21
881 id_ICTRP: NTR4108
-----------------------
882 id_EU 2013-000901-21
882 id_ICTRP: NTR4486
-----------------

980 id_ICTRP: JPRN-JapicCTI-142708
-----------------------
981 id_EU 2013-004625-81
981 id_ICTRP: PER-045-15
-----------------------
982 id_EU 2013-004625-81
982 id_ICTRP: PER-083-13
-----------------------
983 id_EU 2013-004635-69
983 id_ICTRP: NTR5110
-----------------------
984 id_EU 2013-004635-69
984 id_ICTRP: NTR5317
-----------------------
985 id_EU 2013-004805-14
985 id_ICTRP: NTR4770
-----------------------
986 id_EU 2013-004813-41
986 id_ICTRP: CTRI/2014/09/005065
-----------------------
987 id_EU 2013-004918-18
987 id_ICTRP: NTR4499
-----------------------
988 id_EU 2013-004952-39
988 id_ICTRP: NTR4893
-----------------------
989 id_EU 2013-005134-38
989 id_ICTRP: ChiCTR-ONC-14004365
-----------------------
990 id_EU 2013-005157-75
990 id_ICTRP: NTR5350
-----------------------
991 id_EU 2013-005323-17
991 id_ICTRP: NTR4623
-----------------------
992 id_EU 2013-005355-32
992 id_ICTRP: NTR4494
-----------------------
993 id_EU 2013-005366-19
993 id_ICTRP: PER-006-15
---------

1088 id_ICTRP: CTRI/2016/01/006522
-----------------------
1089 id_EU 2014-003932-38
1089 id_ICTRP: NTR5820
-----------------------
1090 id_EU 2014-004094-17
1090 id_ICTRP: NTR5100
-----------------------
1091 id_EU 2014-004265-25
1091 id_ICTRP: NTR5221
-----------------------
1092 id_EU 2014-004432-18
1092 id_ICTRP: NTR5115
-----------------------
1093 id_EU 2014-004445-26
1093 id_ICTRP: ACTRN12614000870651
-----------------------
1094 id_EU 2014-004579-22
1094 id_ICTRP: PER-065-15
-----------------------
1095 id_EU 2014-004580-20
1095 id_ICTRP: PER-065-15
-----------------------
1096 id_EU 2014-004621-40
1096 id_ICTRP: NTR4906
-----------------------
1097 id_EU 2014-004735-39
1097 id_ICTRP: NTR5041
-----------------------
1098 id_EU 2014-004739-38
1098 id_ICTRP: NTR5106
-----------------------
1099 id_EU 2014-004940-36
1099 id_ICTRP: JPRN-JapicCTI-142499
-----------------------
1100 id_EU 2014-005169-63
1100 id_ICTRP: PER-075-15
-----------------------
1101 id_EU 2014-005217-24
1101 

1192 id_EU 2015-005356-99
1192 id_ICTRP: CTRI/2017/08/009450
-----------------------
1193 id_EU 2015-005356-99
1193 id_ICTRP: CTRI/2017/09/009591
-----------------------
1194 id_EU 2015-005357-12
1194 id_ICTRP: CTRI/2017/08/009450
-----------------------
1195 id_EU 2015-005418-31
1195 id_ICTRP: PER-049-16
-----------------------
1196 id_EU 2015-005419-33
1196 id_ICTRP: PER-049-16
-----------------------
1197 id_EU 2015-005437-53
1197 id_ICTRP: NTR5584
-----------------------
1198 id_EU 2016-002906-38
1198 id_ICTRP: NTR6138
-----------------------
1199 id_EU 2016-003086-25
1199 id_ICTRP: NTR6093
-----------------------
1200 id_EU 2016-003133-18
1200 id_ICTRP: NTR6072
-----------------------
1201 id_EU 2016-003245-29
1201 id_ICTRP: NTR6135
-----------------------
1202 id_EU 2016-003253-15
1202 id_ICTRP: NTR6709
-----------------------
1203 id_EU 2016-003288-20
1203 id_ICTRP: JPRN-JapicCTI-142499
-----------------------
1204 id_EU 2016-003288-20
1204 id_ICTRP: JPRN-JapicCTI-173593
-------

In [26]:
#Sponsor Names
EUprim_ICTRPprim_sponsor_name = list_sponsor_names(list_EU_prim_sponsor_name, list_ICTRP_sponsor_name)
EUsec1_ICTRPprim_sponsor_name = list_sponsor_names(list_EU_sec_sponsor1_name, list_ICTRP_sponsor_name)
EUsec2_ICTRPprim_sponsor_name = list_sponsor_names(list_EU_sec_sponsor2_name, list_ICTRP_sponsor_name)
EUsec3_ICTRPprim_sponsor_name = list_sponsor_names(list_EU_sec_sponsor3_name, list_ICTRP_sponsor_name)

EUprim_ICTRPprim_sponsor_name[0:2]

[0.72, 0.84211]

In [27]:
#Sponsor Types
EUprim_ICTRPprim_sponsor_type = list_sponsor_types(list_EU_prim_sponsor_type, list_ICTRP_sponsor_type)
EUsec1_ICTRPprim_sponsor_type = list_sponsor_types(list_EU_sec_sponsor1_type, list_ICTRP_sponsor_type)
EUsec2_ICTRPprim_sponsor_type = list_sponsor_types(list_EU_sec_sponsor2_type, list_ICTRP_sponsor_type)
EUsec3_ICTRPprim_sponsor_type = list_sponsor_types(list_EU_sec_sponsor3_type, list_ICTRP_sponsor_type)

EUsec2_ICTRPprim_sponsor_type[0:2]

[nan, nan]

In [28]:
#Enrolments
EU_ICTRP_enrol = list_enrolments(list_EU_enrol, list_ICTRP_enrol)
EU_ICTRP_enrol[0:2]

[560.0, 207.0]

In [29]:
#Save the final dataframe

df_eu_ictrp['enrolment_EU'] = list_EU_enrol
df_eu_ictrp['enrolment_ICTRP'] = list_ICTRP_enrol
df_eu_ictrp['enrolment_diff'] = EU_ICTRP_enrol


df_eu_ictrp['spon_ICTRP'] = list_ICTRP_sponsor_name

df_eu_ictrp['primary_spon_EU'] = list_EU_prim_sponsor_name
df_eu_ictrp['prim_spon_similarity'] = EUprim_ICTRPprim_sponsor_name

df_eu_ictrp['secondary_spon1_EU'] = list_EU_sec_sponsor1_name
df_eu_ictrp['sec_spon_similarity1'] = EUsec1_ICTRPprim_sponsor_name

df_eu_ictrp['secondary_spon2_EU'] = list_EU_sec_sponsor2_name
df_eu_ictrp['sec_spon_similarity2'] = EUsec2_ICTRPprim_sponsor_name

df_eu_ictrp['secondary_spon3_EU'] = list_EU_sec_sponsor3_name
df_eu_ictrp['sec_spon_similarity3'] = EUsec3_ICTRPprim_sponsor_name



df_eu_ictrp['spon_type_ICTRP'] = list_ICTRP_sponsor_type

df_eu_ictrp['primary_spon_type_EU'] = list_EU_prim_sponsor_type
df_eu_ictrp['same_prim_spon_type'] = EUprim_ICTRPprim_sponsor_type

df_eu_ictrp['secondary_spon1_type_EU'] = list_EU_sec_sponsor1_type
df_eu_ictrp['same_sec_spon_type1'] = EUsec1_ICTRPprim_sponsor_type

df_eu_ictrp['secondary_spon2_type_EU'] = list_EU_sec_sponsor2_type
df_eu_ictrp['same_sec_spon_type2'] = EUsec2_ICTRPprim_sponsor_type

df_eu_ictrp['secondary_spon3_type_EU'] = list_EU_sec_sponsor3_type
df_eu_ictrp['same_sec_spon_type3'] = EUsec3_ICTRPprim_sponsor_type


In [30]:
df_eu_ictrp[:2]

,id_EU,year_EU,long_title_EU,id_ICTRP,year_ICTRP,long_title_ICTRP,title_similarity,enrolment_EU,enrolment_ICTRP,enrolment_diff,...,sec_spon_similarity3,spon_type_ICTRP,primary_spon_type_EU,same_prim_spon_type,secondary_spon1_type_EU,same_sec_spon_type1,secondary_spon2_type_EU,same_sec_spon_type2,secondary_spon3_type_EU,same_sec_spon_type3
110,2004-000029-31,2005.0,"A Randomized, Double-Blind, Placebo-Controlled...",PER-071-11,NaN,"A RANDOMIZED, DOUBLE-BLIND, PLACEBO- CONTROLLE...",0.868217,600.0,40.0,560.0,...,NaN,Commercial,Commercial,1.0,NaN,NaN,NaN,NaN,NaN,NaN
331,2004-000107-18,2006.0,A RANDOMISED OPEN COMPARISON OF INTRAVAGINAL A...,ISRCTN27617624,2005.0,A randomised open comparision of intravaginal ...,0.976378,415.0,622.0,207.0,...,NaN,Commercial,Commercial,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
len(df_eu_ictrp)

1249

In [32]:
df_eu_ictrp.to_excel('../_results/02-sponsors_enrolments/NEW_final_eu_ictrp.xlsx')

## 03- Working for ICTRP and US pairs

#### ICTRP_US (ltitles_years) DATA

In [23]:
df_ictrp_us_ltitles_years = pd.read_excel('../_results/01-years_ltitles/new excels/ictrp_us_years_ltitles_list.xlsx')
df_ictrp_us = df_ictrp_us_ltitles_years.rename(index=str, columns={"similarity": "title_similarity"})
df_ictrp_us[0:3]

,id_ICTRP,year_ICTRP,long_title_ICTRP,id_US,year_US,long_title_US,title_similarity
0,ACTRN12605000059662,2005.0,"Multicentre, Unblinded, Randomised, Controlled...",NCT00221013,2005.0,"Multicentre, Unblinded, Open Label, Randomised...",0.975369
1,ACTRN12605000111673,2004.0,HPV DNA testing versus conventional management...,NCT00119509,2004.0,HPV DNA Testing Versus Conventional Management...,1.000000
2,ACTRN12605000116628,2005.0,The effect of 6 month high intensity progressi...,NCT00465660,2005.0,The Effect of 6 Month High Intensity Progressi...,1.000000


In [24]:
len(df_ictrp_us)

3720

In [34]:
#Create lists of enrolment, sponsor type & name for the pairs 
list_US_sponsor_type = []
list_ICTRP_sponsor_type = []
list_US_sponsor_name = []
list_ICTRP_sponsor_name = []
list_US_enrol = []
list_ICTRP_enrol = []

for i in range(0,len(df_ictrp_us)):
    id_US = df_ictrp_us.id_US[i]
    id_ICTRP = df_ictrp_us.id_ICTRP[i]
    
    for j in range(0,len(df_us)): 
        if (id_US == df_us.trial_id[j]) : 
            
            list_US_sponsor_type.append(df_us.sponsor_type[j])
            list_US_sponsor_name.append(df_us.primary_spons[j])
            list_US_enrol.append(df_us.enrolment[j])
            
            print(i,"id_US",id_US)
            
    for k in range(0,len(df_ictrp)):
        if (id_ICTRP == df_ictrp.trial_id[k]) :
            
            list_ICTRP_sponsor_type.append(df_ictrp.sponsor_type[k])
            list_ICTRP_sponsor_name.append(df_ictrp.primary_spons[k])
            list_ICTRP_enrol.append(df_ictrp.enrolment[k])
            
            print(i,"id_ICTRP:",id_ICTRP)
            print('-----------------------')


0 id_US NCT00221013
0 id_ICTRP: ACTRN12605000059662
-----------------------
1 id_US NCT00119509
1 id_ICTRP: ACTRN12605000111673
-----------------------
2 id_US NCT00465660
2 id_ICTRP: ACTRN12605000116628
-----------------------
3 id_US NCT00126308
3 id_ICTRP: ACTRN12605000132640
-----------------------
4 id_US NCT00685828
4 id_ICTRP: ACTRN12605000138684
-----------------------
5 id_US NCT00009906
5 id_ICTRP: ACTRN12605000138684
-----------------------
6 id_US NCT00273260
6 id_ICTRP: ACTRN12605000139673
-----------------------
7 id_US NCT00763256
7 id_ICTRP: ACTRN12605000260628
-----------------------
8 id_US NCT00139048
8 id_ICTRP: ACTRN12605000293662
-----------------------
9 id_US NCT00226876
9 id_ICTRP: ACTRN12605000307606
-----------------------
10 id_US NCT00091819
10 id_ICTRP: ACTRN12605000323628
-----------------------
11 id_US NCT00107978
11 id_ICTRP: ACTRN12605000323628
-----------------------
12 id_US NCT00124020
12 id_ICTRP: ACTRN12605000324617
-----------------------
13 id_

105 id_ICTRP: ACTRN12607000541404
-----------------------
106 id_US NCT00548587
106 id_ICTRP: ACTRN12607000544471
-----------------------
107 id_US NCT00620035
107 id_ICTRP: ACTRN12607000556448
-----------------------
108 id_US NCT00412919
108 id_ICTRP: ACTRN12607000584437
-----------------------
109 id_US NCT00664378
109 id_ICTRP: ACTRN12607000584437
-----------------------
110 id_US NCT00975793
110 id_ICTRP: ACTRN12608000053325
-----------------------
111 id_US NCT00650949
111 id_ICTRP: ACTRN12608000158369
-----------------------
112 id_US NCT00640302
112 id_ICTRP: ACTRN12608000176369
-----------------------
113 id_US NCT00641641
113 id_ICTRP: ACTRN12608000177358
-----------------------
114 id_US NCT00640211
114 id_ICTRP: ACTRN12608000178347
-----------------------
115 id_US NCT00640874
115 id_ICTRP: ACTRN12608000178347
-----------------------
116 id_US NCT00640211
116 id_ICTRP: ACTRN12608000179336
-----------------------
117 id_US NCT00640874
117 id_ICTRP: ACTRN12608000179336
------

208 id_US NCT02272569
208 id_ICTRP: ACTRN12613001102763
-----------------------
209 id_US NCT02007421
209 id_ICTRP: ACTRN12613001335785
-----------------------
210 id_US NCT02001844
210 id_ICTRP: ACTRN12613001336774
-----------------------
211 id_US NCT02044549
211 id_ICTRP: ACTRN12614000117617
-----------------------
212 id_US NCT02385201
212 id_ICTRP: ACTRN12614000153617
-----------------------
213 id_US NCT02143349
213 id_ICTRP: ACTRN12614000305628
-----------------------
214 id_US NCT02788552
214 id_ICTRP: ACTRN12614000327684
-----------------------
215 id_US NCT02528903
215 id_ICTRP: ACTRN12614000359639
-----------------------
216 id_US NCT02255968
216 id_ICTRP: ACTRN12614000359639
-----------------------
217 id_US NCT01877785
217 id_ICTRP: ACTRN12614000359639
-----------------------
218 id_US NCT02366364
218 id_ICTRP: ACTRN12614000359639
-----------------------
219 id_US NCT02228707
219 id_ICTRP: ACTRN12614000359639
-----------------------
220 id_US NCT02596399
220 id_ICTRP: ACTR

311 id_US NCT02859701
311 id_ICTRP: ACTRN12616000049471
-----------------------
312 id_US NCT02538874
312 id_ICTRP: ACTRN12616000049471
-----------------------
313 id_US NCT02903095
313 id_ICTRP: ACTRN12616000049471
-----------------------
314 id_US NCT03335553
314 id_ICTRP: ACTRN12616000049471
-----------------------
315 id_US NCT02790125
315 id_ICTRP: ACTRN12616000049471
-----------------------
316 id_US NCT02415439
316 id_ICTRP: ACTRN12616000049471
-----------------------
317 id_US NCT03237156
317 id_ICTRP: ACTRN12616000049471
-----------------------
318 id_US NCT02954263
318 id_ICTRP: ACTRN12616000049471
-----------------------
319 id_US NCT02709928
319 id_ICTRP: ACTRN12616000049471
-----------------------
320 id_US NCT02711462
320 id_ICTRP: ACTRN12616000049471
-----------------------
321 id_US NCT02657122
321 id_ICTRP: ACTRN12616000049471
-----------------------
322 id_US NCT03294707
322 id_ICTRP: ACTRN12616000049471
-----------------------
323 id_US NCT02616445
323 id_ICTRP: ACTR

414 id_US NCT02860715
414 id_ICTRP: ACTRN12616001156471
-----------------------
415 id_US NCT02616445
415 id_ICTRP: ACTRN12616001156471
-----------------------
416 id_US NCT02639078
416 id_ICTRP: ACTRN12616001156471
-----------------------
417 id_US NCT02898727
417 id_ICTRP: ACTRN12616001265460
-----------------------
418 id_US NCT02877966
418 id_ICTRP: ACTRN12616001402437
-----------------------
419 id_US NCT02578134
419 id_ICTRP: ACTRN12616001430426
-----------------------
420 id_US NCT02528903
420 id_ICTRP: ACTRN12616001438448
-----------------------
421 id_US NCT03199313
421 id_ICTRP: ACTRN12616001438448
-----------------------
422 id_US NCT02958436
422 id_ICTRP: ACTRN12616001438448
-----------------------
423 id_US NCT02596399
423 id_ICTRP: ACTRN12616001438448
-----------------------
424 id_US NCT02777151
424 id_ICTRP: ACTRN12616001438448
-----------------------
425 id_US NCT02086019
425 id_ICTRP: ACTRN12616001463460
-----------------------
426 id_US NCT02353494
426 id_ICTRP: ACTR

...

3242 id_US NCT02452047
3242 id_ICTRP: PER-026-15
-----------------------
3243 id_US NCT00515112
3243 id_ICTRP: PER-027-11
-----------------------
3244 id_US NCT01234311
3244 id_ICTRP: PER-027-11
-----------------------
3245 id_US NCT01946204
3245 id_ICTRP: PER-027-11
-----------------------
3246 id_US NCT02057666
3246 id_ICTRP: PER-027-11
-----------------------
3247 id_US NCT01451411
3247 id_ICTRP: PER-027-12
-----------------------
3248 id_US NCT01966471
3248 id_ICTRP: PER-027-14
-----------------------
3249 id_US NCT02366143
3249 id_ICTRP: PER-027-15
-----------------------
3250 id_US NCT02810457
3250 id_ICTRP: PER-027-16
-----------------------
3251 id_US NCT01232569
3251 id_ICTRP: PER-028-11
-----------------------
3252 id_US NCT01523587
3252 id_ICTRP: PER-028-12
-----------------------
3253 id_US NCT01748929
3253 id_ICTRP: PER-028-13
-----------------------
3254 id_US NCT01968733
3254 id_ICTRP: PER-028-14
-----------------------
3255 id_US NCT02367794
3255 id_ICTRP: PER-028-15
--

3354 id_ICTRP: PER-048-12
-----------------------
3355 id_US NCT01841697
3355 id_ICTRP: PER-048-13
-----------------------
3356 id_US NCT01682759
3356 id_ICTRP: PER-048-13
-----------------------
3357 id_US NCT01485614
3357 id_ICTRP: PER-048-13
-----------------------
3358 id_US NCT02216123
3358 id_ICTRP: PER-048-14
-----------------------
3359 id_US NCT02437890
3359 id_ICTRP: PER-048-15
-----------------------
3360 id_US NCT01332019
3360 id_ICTRP: PER-049-11
-----------------------
3361 id_US NCT01480479
3361 id_ICTRP: PER-049-12
-----------------------
3362 id_US NCT02084602
3362 id_ICTRP: PER-049-13
-----------------------
3363 id_US NCT02477826
3363 id_ICTRP: PER-049-15
-----------------------
3364 id_US NCT02792231
3364 id_ICTRP: PER-049-16
-----------------------
3365 id_US NCT02792218
3365 id_ICTRP: PER-049-16
-----------------------
3366 id_US NCT01868061
3366 id_ICTRP: PER-050-13
-----------------------
3367 id_US NCT01867125
3367 id_ICTRP: PER-050-13
-----------------------
3

3467 id_US NCT01442376
3467 id_ICTRP: PER-068-11
-----------------------
3468 id_US NCT01488708
3468 id_ICTRP: PER-068-12
-----------------------
3469 id_US NCT01782326
3469 id_ICTRP: PER-068-13
-----------------------
3470 id_US NCT02021318
3470 id_ICTRP: PER-068-14
-----------------------
3471 id_US NCT02174627
3471 id_ICTRP: PER-068-14
-----------------------
3472 id_US NCT02564263
3472 id_ICTRP: PER-068-15
-----------------------
3473 id_US NCT02178592
3473 id_ICTRP: PER-069-14
-----------------------
3474 id_US NCT02528188
3474 id_ICTRP: PER-069-15
-----------------------
3475 id_US NCT02725411
3475 id_ICTRP: PER-069-15
-----------------------
3476 id_US NCT02709486
3476 id_ICTRP: PER-069-15
-----------------------
3477 id_US NCT00790205
3477 id_ICTRP: PER-070-11
-----------------------
3478 id_US NCT01144338
3478 id_ICTRP: PER-070-11
-----------------------
3479 id_US NCT01634152
3479 id_ICTRP: PER-070-12
-----------------------
3480 id_US NCT01634139
3480 id_ICTRP: PER-070-12
--

3579 id_ICTRP: PER-102-12
-----------------------
3580 id_US NCT01524783
3580 id_ICTRP: PER-113-12
-----------------------
3581 id_US NCT01682512
3581 id_ICTRP: PER-114-12
-----------------------
3582 id_US NCT01444430
3582 id_ICTRP: PER-115-11
-----------------------
3583 id_US NCT01419717
3583 id_ICTRP: PER-116-11
-----------------------
3584 id_US NCT01641939
3584 id_ICTRP: PER-118-12
-----------------------
3585 id_US NCT01516736
3585 id_ICTRP: PER-119-11
-----------------------
3586 id_US NCT01644643
3586 id_ICTRP: PER-119-12
-----------------------
3587 id_US NCT01467713
3587 id_ICTRP: PER-122-12
-----------------------
3588 id_US NCT01685151
3588 id_ICTRP: PER-122-12
-----------------------
3589 id_US NCT01563029
3589 id_ICTRP: PER-124-11
-----------------------
3590 id_US NCT01573767
3590 id_ICTRP: PER-125-11
-----------------------
3591 id_US NCT01879410
3591 id_ICTRP: PER-125-12
-----------------------
3592 id_US NCT01822899
3592 id_ICTRP: PER-125-12
-----------------------
3

3691 id_US NCT02286453
3691 id_ICTRP: TCTR20140816001
-----------------------
3692 id_US NCT02766179
3692 id_ICTRP: TCTR20140901002
-----------------------
3693 id_US NCT02629835
3693 id_ICTRP: TCTR20150624001
-----------------------
3694 id_US NCT02568059
3694 id_ICTRP: TCTR20150917001
-----------------------
3695 id_US NCT02576808
3695 id_ICTRP: TCTR20151001001
-----------------------
3696 id_US NCT02862509
3696 id_ICTRP: TCTR20151103001
-----------------------
3697 id_US NCT02618785
3697 id_ICTRP: TCTR20151121001
-----------------------
3698 id_US NCT02684942
3698 id_ICTRP: TCTR20160208002
-----------------------
3699 id_US NCT02787707
3699 id_ICTRP: TCTR20160401004
-----------------------
3700 id_US NCT02752217
3700 id_ICTRP: TCTR20160405001
-----------------------
3701 id_US NCT02768428
3701 id_ICTRP: TCTR20160426002
-----------------------
3702 id_US NCT02914106
3702 id_ICTRP: TCTR20160510004
-----------------------
3703 id_US NCT02568059
3703 id_ICTRP: TCTR20160729002
----------

In [35]:
list_US_id = ['NCT00119509','NCT00465660','NCT00126308','NCT00273260','NCT00119509']

In [36]:
df3 = df_us[df_us['trial_id'].isin(list_US_id)] 
df3[:2]

,trial_id,primary_spons,sponsor_type,enrolment
36593,NCT00465660,University of Sydney,Non-Commercial,60.0
90113,NCT00273260,Menzies Institute for Medical Research,Non-Commercial,400.0


In [37]:
ICTRP_US_sponsor_type = list_sponsor_types(list_US_sponsor_type, list_ICTRP_sponsor_type)
ICTRP_US_sponsor_type[0:2]

[1, 1]

In [38]:
ICTRP_US_sponsor_name = list_sponsor_names(list_US_sponsor_name, list_ICTRP_sponsor_name)
ICTRP_US_sponsor_name[0:2]

[0.86957, 0.78431]

In [39]:
ICTRP_US_enrol = list_enrolments(list_US_enrol, list_ICTRP_enrol)
ICTRP_US_enrol[0:2]

[8.0, 0.0]

In [40]:
#Save the final dataframe
df_ictrp_us['spon_type_ICTRP'] = list_ICTRP_sponsor_type
df_ictrp_us['spon_type_US'] = list_US_sponsor_type
df_ictrp_us['same_spon_type'] = ICTRP_US_sponsor_type

df_ictrp_us['sponsor_ICTRP'] = list_ICTRP_sponsor_name
df_ictrp_us['sponsor_US'] = list_US_sponsor_name
df_ictrp_us['spon_similarity'] = ICTRP_US_sponsor_name

df_ictrp_us['enrolment_ICTRP'] = list_ICTRP_enrol
df_ictrp_us['enrolment_US'] = list_US_enrol
df_ictrp_us['enrolment_diff'] = ICTRP_US_enrol

In [41]:
df_ictrp_us[0:2]

,id_ICTRP,year_ICTRP,long_title_ICTRP,id_US,year_US,long_title_US,title_similarity,spon_type_ICTRP,spon_type_US,same_spon_type,sponsor_ICTRP,sponsor_US,spon_similarity,enrolment_ICTRP,enrolment_US,enrolment_diff
0,ACTRN12605000059662,2005.0,"Multicentre, Unblinded, Randomised, Controlled...",NCT00221013,2005.0,"Multicentre, Unblinded, Open Label, Randomised...",0.975369,Non-Commercial,Non-Commercial,1.0,Other The George Institute,The George Institute,0.86957,1500.0,1508.0,8.0
1,ACTRN12605000111673,2004.0,HPV DNA testing versus conventional management...,NCT00119509,2004.0,HPV DNA Testing Versus Conventional Management...,1.000000,Non-Commercial,Non-Commercial,1.0,University University of Sydney,University of Sydney,0.78431,300.0,300.0,0.0


In [42]:
df_ictrp_us.to_excel('../_results/02-sponsors_enrolments/NEW_final_ictrp_us.xlsx')

### Save the final pickles

In [23]:
with open('../_results/02-sponsors_enrolments/final_eu_us.pickle', 'wb') as f:
    pickle.dump(df_eu_us, f) 

In [24]:
with open('../_results/02-sponsors_enrolments/final_eu_ictrp.pickle', 'wb') as f:
    pickle.dump(df_eu_ictrp, f) 

In [ ]:
with open('../_results/02-sponsors_enrolments/final_ictrp_us.pickle', 'wb') as f:
    pickle.dump(df_ictrp_us, f) 